# Bitcoin RL Trading Bot - Cloud Training

Bu notebook Google Colab'da GPU ile hızlı training yapar.

**Adımlar:**
1. GPU kontrol et
2. Repo'yu clone'la
3. Dependencies kur
4. Training başlat
5. Modeli indir

## 1. GPU Kontrol

In [ ]:
# GPU kontrol
!nvidia-smi

import torch
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. Repo Clone

In [ ]:
# Repo'yu clone'la
!git clone https://github.com/barturker/bitcoin-bot.git
%cd bitcoin-bot

## 3. Dependencies Kur

In [ ]:
# Dependencies kur
!pip install -q stable-baselines3[extra] pandas numpy matplotlib scikit-learn pandas_ta pyyaml gymnasium

## 4. Data Yükle

**ÖNEMLİ:** `btcusd_1-min_data.csv` dosyasını yüklemeniz gerekiyor.

Seçenek A: Google Drive'dan
Seçenek B: Doğrudan upload

In [ ]:
# Seçenek A: Google Drive'dan (dosyayı Drive'a yüklediysen)
from google.colab import drive
drive.mount('/content/drive')

# Drive'daki dosyayı kopyala (path'i kendi dosyana göre değiştir)
!cp "/content/drive/MyDrive/btcusd_1-min_data.csv" data/

In [ ]:
# Seçenek B: Doğrudan upload (yukarıdaki hücreyi çalıştırma)
from google.colab import files
import shutil

print("btcusd_1-min_data.csv dosyasını yükleyin:")
uploaded = files.upload()

# data klasörüne taşı
for filename in uploaded.keys():
    shutil.move(filename, f'data/{filename}')
    print(f"Moved {filename} to data/")

In [ ]:
# Data kontrol
!ls -la data/
!head -5 data/btcusd_1-min_data.csv

## 5. Config Ayarla (Opsiyonel)

Training parametrelerini değiştirmek istersen:

In [ ]:
import yaml

# Config'i oku
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Parametreleri değiştir (opsiyonel)
config['model']['total_timesteps'] = 500000  # İstediğin değer
config['model']['batch_size'] = 256  # GPU için daha büyük batch
config['model']['n_steps'] = 2048

# Kaydet
with open('config.yaml', 'w') as f:
    yaml.dump(config, f, default_flow_style=False)

print("Config güncellendi:")
print(f"  Total timesteps: {config['model']['total_timesteps']}")
print(f"  Batch size: {config['model']['batch_size']}")
print(f"  N steps: {config['model']['n_steps']}")

## 6. Cache Temizle

Eski scaler varsa temizle (feature sayısı değişmiş olabilir)

In [ ]:
# Cache temizle
!rm -rf cache/*
!mkdir -p cache logs/trades logs/tensorboard models
print("Cache temizlendi, klasörler oluşturuldu.")

## 7. Training Başlat

In [ ]:
# Training başlat
%cd /content/bitcoin-bot
!python src/train.py

## 8. Sonuçları Görüntüle

In [ ]:
# Eğitilmiş modelleri listele
!ls -la models/

# Equity curve'ları göster
from IPython.display import Image, display
import glob

for img in glob.glob('logs/*_equity_curve.png'):
    print(f"\n{img}:")
    display(Image(img))

## 9. Modeli İndir

In [ ]:
# En son modeli indir
from google.colab import files
import glob

# En son final modeli bul
models = sorted(glob.glob('models/ppo_btc_final_*.zip'))
if models:
    latest_model = models[-1]
    print(f"İndiriliyor: {latest_model}")
    files.download(latest_model)
else:
    print("Final model bulunamadı, tüm modeller:")
    !ls -la models/

In [ ]:
# Best model'i de indir (EvalCallback'den)
if os.path.exists('models/best_model.zip'):
    files.download('models/best_model.zip')
    print("best_model.zip indirildi")

In [ ]:
# Scaler'ı da indir (paper trading için lazım)
if os.path.exists('cache/scaler.pkl'):
    files.download('cache/scaler.pkl')
    print("scaler.pkl indirildi")

## 10. Google Drive'a Kaydet (Opsiyonel)

Modelleri Drive'a kaydet, böylece kaybolmaz.

In [ ]:
# Drive'a kaydet
!cp -r models/ "/content/drive/MyDrive/bitcoin-bot-models/"
!cp cache/scaler.pkl "/content/drive/MyDrive/bitcoin-bot-models/"
print("Modeller Google Drive'a kaydedildi!")

---

## Bitti!

İndirdiğin model dosyasını (`ppo_btc_final_*.zip`) projenin `models/` klasörüne koy.

Sonra dashboard'dan backtest yapabilirsin.